**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Sep  2 19:05:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 catkin_ws
'Colab Notebooks'
'kws-fpod-yaq - 23 Jul 2022 (1).gjam'
'kws-fpod-yaq - 23 Jul 2022.gjam'
 models
'My Drive'
'sqd-ndqq-txh - 11 Agu 2022.gjam'
 Storage
'Untitled Diagram-database-GUI.drawio.png'
'Untitled Diagram.drawio'
'Untitled Diagram-Raspy-database.drawio.png'
 yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1150

**3) Configure Darknet network for training YOLO V3**

In [5]:
# !cp cfg/yolov3.cfg cfg/yolov3_training.cfg

# Untuk Tiny
!cp cfg/yolov3-tiny.cfg cfg/yolov3-tiny_training.cfg

In [6]:
# !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
# !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# !sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
# !sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# !sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
# !sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
# !sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

# Untuk Tiny
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3-tiny_training.cfg
!sed -i '135 s@classes=80@classes=1@' cfg/yolov3-tiny_training.cfg
!sed -i '177 s@classes=80@classes=1@' cfg/yolov3-tiny_training.cfg
!sed -i '127 s@filters=255@filters=18@' cfg/yolov3-tiny_training.cfg
!sed -i '171 s@filters=255@filters=18@' cfg/yolov3-tiny_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "bola" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [9]:
# Download weights darknet model 53
# !wget https://pjreddie.com/media/files/darknet53.conv.74

# Untuk Tiny
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2022-09-02 19:06:49--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights.3’

yolov3-tiny.weights 100%[===================>]  33.79M  14.0MB/s    in 2.4s    

2022-09-02 19:06:52 (14.0 MB/s) - ‘yolov3-tiny.weights.3’ saved [35434956/35434956]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/pics/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/pics/1.jpg     
  inflating: data/obj/pics/1.txt     
  inflating: data/obj/pics/10.jpg    
  inflating: data/obj/pics/10.txt    
  inflating: data/obj/pics/11.jpg    
  inflating: data/obj/pics/11.txt    
  inflating: data/obj/pics/12.jpg    
  inflating: data/obj/pics/12.txt    
  inflating: data/obj/pics/13.jpg    
  inflating: data/obj/pics/13.txt    
  inflating: data/obj/pics/14.jpg    
  inflating: data/obj/pics/14.txt    
  inflating: data/obj/pics/15.jpg    
  inflating: data/obj/pics/15.txt    
  inflating: data/obj/pics/16.jpg    
  inflating: data/obj/pics/16.txt    
  inflating: data/obj/pics/17.jpg    
  inflating: data/obj/pics/17.txt    
  inflating: data/obj/pics/18.jpg    
  inflating: data/obj/pics/18.txt    
  inflating: data/obj/pics/19.jpg    
  inflating: data/obj/pics/19.txt    
  inflating: data/obj/pics/2.jpg     
  inflating: data/ob

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/pics/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.609896 0.504167 0.515625 0.683333

['15', '0.609896', '0.504167', '0.515625', '0.683333']
0 data/obj/pics/2022-09-02-214412_2 (copy).txt
0 0.609896 0.504167 0.515625 0.683333
15 0.516146 0.473611 0.509375 0.680556

['15', '0.516146', '0.473611', '0.509375', '0.680556']
1 data/obj/pics/5.txt
0 0.516146 0.473611 0.509375 0.680556
15 0.499479 0.418056 0.601042 0.805556

['15', '0.499479', '0.418056', '0.601042', '0.805556']
2 data/obj/pics/2022-09-02-214445_2.txt
0 0.499479 0.418056 0.601042 0.805556
15 0.432812 0.568750 0.605208 0.806944

['15', '0.432812', '0.568750', '0.605208', '0.806944']
3 data/obj/pics/2022-09-02-214356_2.txt
0 0.432812 0.568750 0.605208 0.806944
15 0.659896 0.489583 0.680208 0.901389

['15', '0.659896', '0.489583', '0.680208', '0.901389']
4 data/obj/pics/27.txt
0 0.659896 0.489583 0.680208 0.901389
15 0.527083 0.546528 0.504167 0.670833

['15', '0.527083', '0.546528', '0.504167', '0.670833']
5 data/obj/pics/2022-09-02-214356_3 (copy).txt
0 0.527083 0.546528 0

In [12]:
import glob
images_list = glob.glob("data/obj/pics/*.jpg")
print(images_list)

['data/obj/pics/26.jpg', 'data/obj/pics/22.jpg', 'data/obj/pics/4.jpg', 'data/obj/pics/11.jpg', 'data/obj/pics/37.jpg', 'data/obj/pics/2022-09-02-214341_1.jpg', 'data/obj/pics/27.jpg', 'data/obj/pics/2022-09-02-214356_1 (copy).jpg', 'data/obj/pics/8.jpg', 'data/obj/pics/18.jpg', 'data/obj/pics/35.jpg', 'data/obj/pics/2022-09-02-214341_2.jpg', 'data/obj/pics/2022-09-02-214356_4.jpg', 'data/obj/pics/33.jpg', 'data/obj/pics/2022-09-02-214356_2 (copy).jpg', 'data/obj/pics/15.jpg', 'data/obj/pics/2.jpg', 'data/obj/pics/2022-09-02-214341_3.jpg', 'data/obj/pics/16.jpg', 'data/obj/pics/2022-09-02-214356_1.jpg', 'data/obj/pics/2022-09-02-214412_4.jpg', 'data/obj/pics/2022-09-02-214445_3 (copy).jpg', 'data/obj/pics/28.jpg', 'data/obj/pics/2022-09-02-214412_3.jpg', 'data/obj/pics/2022-09-02-214356_2.jpg', 'data/obj/pics/2022-09-02-214445_2 (copy).jpg', 'data/obj/pics/2022-09-02-214445_3.jpg', 'data/obj/pics/39.jpg', 'data/obj/pics/2022-09-02-214428_3 (copy).jpg', 'data/obj/pics/2022-09-02-214412_

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [14]:
# Start the training
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# Untuk Tiny
!./darknet partial cfg/yolov3-tiny_training.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15
!./darknet detector train data/obj.data cfg/yolov3-tiny_training.cfg yolov3-tiny.conv.15 -dont_show

Output streaming akan dipotong hingga 5000 baris terakhir.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 121530, rewritten_bbox = 0.000000 % 
]2;1903/2000: loss=0.3 hours left=0.1
 1903: 0.332615, 0.219538 avg loss, 0.001000 rate, 1.394084 seconds, 121792 images, 0.053769 hours left
Loaded: 0.000075 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.703766), count: 4, class_loss = 0.878183, iou_loss = 0.225494, total_loss = 1.103677 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 121534, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.801887), count: 4, class_loss = 0.397124, iou_loss = 0.208345, total_loss = 0.605469